In [1]:
import torch
from torch import nn, optim
import torch.nn.functional as F
from torch.autograd import Variable
from torch.utils.data import DataLoader
from torchvision import transforms
from torchvision import datasets
import time
# 定义超参数
batch_size = 32
learning_rate = 1e-3
num_epoches = 100

# 下载训练集 MNIST 手写数字训练集
train_dataset = datasets.MNIST(
    root='./data', train=True, transform=transforms.ToTensor(), download=True)

test_dataset = datasets.MNIST(
    root='./data', train=False, transform=transforms.ToTensor())

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

In [2]:
train_dataset

Dataset MNIST
    Number of datapoints: 60000
    Split: train
    Root Location: ./data
    Transforms (if any): ToTensor()
    Target Transforms (if any): None

In [3]:
train_loader

In [4]:
class Logstic_Regression(nn.Module):
    def __init__(self, in_dim, n_class):
        super(Logstic_Regression, self).__init__()
        self.logstic = nn.Linear(in_dim, n_class)

    def forward(self, x):
        out = self.logstic(x)
        return out


model = Logstic_Regression(28 * 28, 10)  # 图片大小是28x28
use_gpu = torch.cuda.is_available()  # 判断是否有GPU加速

In [5]:
model

Logstic_Regression(
  (logstic): Linear(in_features=784, out_features=10, bias=True)
)

In [6]:
train_loader

In [7]:
for i, data in enumerate(train_loader, 1):
    img, label = data                # data[0].size()=32,1,28,28  data[1].size()=32
    img = img.view(img.size(0), -1)  # 将图片展开成 28x28 , img.size() = 32*784

In [54]:
i

1875

In [34]:
img

tensor([[0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        ...,
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.]])

In [40]:
img = Variable(img)

In [41]:
img

tensor([[0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        ...,
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.]])

In [43]:
print(label)

tensor([1, 6, 8, 9, 9, 3, 1, 2, 5, 3, 1, 3, 8, 3, 0, 1, 8, 0, 8, 2, 3, 4, 5, 9,
        5, 1, 3, 3, 3, 4, 5, 0])


In [42]:
print(label.size())
print(img.size())

torch.Size([32])
torch.Size([32, 784])


In [36]:
print(data,'\n')
print(data[0].size(),'\n')
print(data[1].size(),'\n')

[tensor([[[[0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
          [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
          [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
          ...,
          [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
          [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
          [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000]]],


        [[[0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
          [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
          [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
          ...,
          [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
          [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
          [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000]]],


        [[[0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
          [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
          [0.0000, 0.0000, 0.0000,  .

In [1]:
if use_gpu:
    model = model.cuda()
# 定义loss和optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=learning_rate)

# 开始训练
#for epoch in range(num_epoches):
for epoch in range(1):
    print('*' * 10)
    print('epoch {}'.format(epoch + 1))
    since = time.time()
    running_loss = 0.0
    running_acc = 0.0
    for i, data in enumerate(train_loader, 1):
        img, label = data
        img = img.view(img.size(0), -1)  # 将图片展开成 28x28
        if use_gpu:
            img = Variable(img).cuda()
            label = Variable(label).cuda()
        else:
            img = Variable(img)
            label = Variable(label)
        # 向前传播
        out = model(img)
        loss = criterion(out, label)
        running_loss += loss.data[0] * label.size(0)
        _, pred = torch.max(out, 1)
        num_correct = (pred == label).sum()
        running_acc += num_correct.data[0]
        # 向后传播
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if i % 300 == 0:
            print('[{}/{}] Loss: {:.6f}, Acc: {:.6f}'.format(
                epoch + 1, num_epoches, running_loss / (batch_size * i),
                running_acc * 100 / (batch_size * i)))
    print('Finish {} epoch, Loss: {:.6f}, Acc: {:.6f}'.format(
        epoch + 1, running_loss / (len(train_dataset)), running_acc *100 / (len(
            train_dataset))))
    model.eval()
    eval_loss = 0.
    eval_acc = 0.
    for data in test_loader:
        img, label = data
        img = img.view(img.size(0), -1)
        if use_gpu:
            img = Variable(img, volatile=True).cuda()
            label = Variable(label, volatile=True).cuda()
        else:
            img = Variable(img, volatile=True)
            label = Variable(label, volatile=True)
        out = model(img)
        loss = criterion(out, label)
        eval_loss += loss.data[0] * label.size(0)
        _, pred = torch.max(out, 1)
        num_correct = (pred == label).sum()
        eval_acc += num_correct.data[0]
    print('Test Loss: {:.6f}, Acc: {:.6f}'.format(eval_loss / (len(
        test_dataset)), eval_acc / (len(test_dataset))))
    print('Time:{:.1f} s'.format(time.time() - since))
    print()

# 保存模型
torch.save(model.state_dict(), './logstic.pth')

NameError: name 'use_gpu' is not defined

In [5]:
__author__ = 'SherlockLiao'

import torch
from torch import nn, optim
import torch.nn.functional as F
from torch.autograd import Variable
from torch.utils.data import DataLoader
from torchvision import transforms
from torchvision import datasets
import time
# 定义超参数
batch_size = 32
learning_rate = 1e-3
num_epoches = 100

# 下载训练集 MNIST 手写数字训练集
train_dataset = datasets.MNIST(
    root='./data', train=True, transform=transforms.ToTensor(), download=True)

test_dataset = datasets.MNIST(
    root='./data', train=False, transform=transforms.ToTensor())

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)


# 定义 Logistic Regression 模型
"""
class Logstic_Regression(nn.Module):
    def __init__(self, in_dim, n_class):
        super(Logstic_Regression, self).__init__()
        #self.logstic = nn.Linear(in_dim, n_class)
        self.test = nn.Linear(in_dim, n_class)

    def forward(self, x):
        out = self.test(x)
        return out
"""
"""
class Logstic_Regression(nn.Module):
    def __init__(self):
        super(Logstic_Regression, self).__init__()
        #self.logstic = nn.Linear(in_dim, n_class)
        self.test = nn.Linear(784, 10)

    def forward(self, x):
        out = self.test(x)
        return out
"""


class Logstic_Regression(nn.Module):
    def __init__(self, in_dim, n_class):
        super(Logstic_Regression, self).__init__()
        #self.logstic = nn.Linear(in_dim, n_class)
        self.test = nn.Linear(in_dim, n_class)

    def forward(self, x):
        x = self.test(x)
        out = F.sigmoid(x)
        return out

model = Logstic_Regression(28*28, 10)  # 图片大小是28x28
#use_gpu = torch.cuda.is_available()  # 判断是否有GPU加速
use_gpu = False

if use_gpu:
    model = model.cuda()
# 定义loss和optimizer
# criterion = nn.CrossEntropyLoss()
criterion = nn.MSELoss()
optimizer = optim.SGD(model.parameters(), lr=learning_rate)

In [6]:
# 开始训练
#for epoch in range(num_epoches):
for epoch in range(5):
    print('*' * 10)
    print('epoch {}'.format(epoch + 1))
    since = time.time()
    running_loss = 0.0
    running_acc = 0.0
    for i, data in enumerate(train_loader, 1):
        img, label = data                # data[0].size()=32,1,28,28  data[1].size()=32
        img = img.view(img.size(0), -1)  # 将图片展开成 28x28 , 32*784
        if use_gpu:
            img = Variable(img).cuda()
            label = Variable(label).cuda()
        else:
            img = Variable(img)
            label = Variable(label)
        # 向前传播
        out = model(img)
        loss = criterion(out, label)
        running_loss += loss.data[0] * label.size(0)
        _, pred = torch.max(out, 1)
        num_correct = (pred == label).sum()
        running_acc += num_correct.data[0]
        # 向后传播
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if i % 300 == 0:
            print('[{}/{}] Loss: {:.6f}, Acc: {:.6f}'.format(
                epoch + 1, num_epoches, running_loss  / (batch_size * i),
                running_acc * 100 / (batch_size * i)))
    print('Finish {} epoch, Loss: {:.6f}, Acc: {:.6f}'.format(
        epoch + 1, running_loss / (len(train_dataset)), running_acc * 100 / (len(
            train_dataset))))
    
    
    model.eval()
    eval_loss = 0.
    eval_acc = 0.
    for data in test_loader:
        img, label = data
        img = img.view(img.size(0), -1)
        if use_gpu:
            img = Variable(img, volatile=True).cuda()
            label = Variable(label, volatile=True).cuda()
        else:
            img = Variable(img, volatile=True)
            label = Variable(label, volatile=True)
        out = model(img)
        loss = criterion(out, label)
        eval_loss += loss.data[0] * label.size(0)
        _, pred = torch.max(out, 1)
        num_correct = (pred == label).sum()
        eval_acc += num_correct.data[0]
    print('Test Loss: {:.6f}, Acc: {:.6f}'.format(eval_loss / (len(
        test_dataset)), eval_acc * 100 / (len(test_dataset))))
    print('Time:{:.1f} s'.format(time.time() - since))
    print()

# 保存模型
torch.save(model.state_dict(), './logstic.pth')

**********
epoch 1


D:\Anaconda3\envs\torch\lib\site-packages\torch\nn\functional.py:1006: UserWarning: nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.
  warnings.warn("nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.")


RuntimeError: Expected object of type torch.FloatTensor but found type torch.LongTensor for argument #2 'target'

In [1]:
__author__ = 'SherlockLiao'

import torch
from torch import nn, optim
import torch.nn.functional as F
from torch.autograd import Variable
from torch.utils.data import DataLoader
from torchvision import transforms
from torchvision import datasets
import time
# 定义超参数
batch_size = 32
learning_rate = 1e-3
num_epoches = 100

# 下载训练集 MNIST 手写数字训练集
train_dataset = datasets.MNIST(
    root='./data', train=True, transform=transforms.ToTensor(), download=True)

test_dataset = datasets.MNIST(
    root='./data', train=False, transform=transforms.ToTensor())

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)


# 定义 Logistic Regression 模型
"""
class Logstic_Regression(nn.Module):
    def __init__(self, in_dim, n_class):
        super(Logstic_Regression, self).__init__()
        #self.logstic = nn.Linear(in_dim, n_class)
        self.test = nn.Linear(in_dim, n_class)

    def forward(self, x):
        out = self.test(x)
        return out
"""

class Logstic_Regression(nn.Module):
    def __init__(self, in_dim, n_class):
        super(Logstic_Regression, self).__init__()
        #self.logstic = nn.Linear(in_dim, n_class)
        self.test = nn.Linear(in_dim, n_class)

    def forward(self, x):
        x = self.test(x)
        out = F.sigmoid(x)
        return out

model = Logstic_Regression(28 * 28, 10)  # 图片大小是28x28
#use_gpu = torch.cuda.is_available()  # 判断是否有GPU加速
use_gpu = False

if use_gpu:
    model = model.cuda()
# 定义loss和optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=learning_rate)

In [12]:
# 开始训练
#for epoch in range(num_epoches):
for epoch in range(2):
    print('*' * 10)
    print('epoch {}'.format(epoch + 1))
    since = time.time()
    running_loss = 0.0
    running_acc = 0.0
    for i, data in enumerate(train_loader, 1):
        img, label = data                # data[0].size()=32,1,28,28  data[1].size()=32
        img = img.view(img.size(0), -1)  # 将图片展开成 28x28 , 32*784
        if use_gpu:
            img = Variable(img).cuda()
            label = Variable(label).cuda()
        else:
            img = Variable(img)
            label = Variable(label)
        # 向前传播
        out = model(img)
        loss = criterion(out, label)
        running_loss += loss.data[0] * label.size(0)
        _, pred = torch.max(out, 1)
        num_correct = (pred == label).sum()
        running_acc += num_correct.data[0]
        # 向后传播
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if i % 300 == 0:
            print('[{}/{}] Loss: {:.3f}, Acc: {:.3f}'.format(
                epoch + 1, num_epoches, running_loss  / (batch_size * i),
                float(running_acc) * 100 / (batch_size * i)))
    print('Finish {} epoch, Loss: {:.3f}, Acc: {:.3f}'.format(
        epoch + 1, running_loss / (len(train_dataset)), float(running_acc) * 100 / (len(
            train_dataset))))
    
    
    model.eval()
    eval_loss = 0.
    eval_acc = 0.
    for data in test_loader:
        img, label = data
        img = img.view(img.size(0), -1)
        if use_gpu:
            img = Variable(img, volatile=True).cuda()
            label = Variable(label, volatile=True).cuda()
        else:
            img = Variable(img, volatile=True)
            label = Variable(label, volatile=True)
        out = model(img)
        loss = criterion(out, label)
        eval_loss += loss.data[0] * label.size(0)
        _, pred = torch.max(out, 1)
        num_correct = (pred == label).sum()
        eval_acc += num_correct.data[0]
    print('Test Loss: {:.3f}, Acc: {:.3f}'.format(eval_loss / (len(
        test_dataset)), float(eval_acc) * 100 / (len(test_dataset))))
    print('Time:{:.1f} s'.format(time.time() - since))
    print()

# 保存模型
torch.save(model.state_dict(), './logstic.pth')

**********
epoch 1


D:\Anaconda3\envs\torch\lib\site-packages\torch\nn\functional.py:1006: UserWarning: nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.
  warnings.warn("nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.")
D:\Anaconda3\envs\torch\lib\site-packages\ipykernel_launcher.py:21: UserWarning: invalid index of a 0-dim tensor. This will be an error in PyTorch 0.5. Use tensor.item() to convert a 0-dim tensor to a Python number
D:\Anaconda3\envs\torch\lib\site-packages\ipykernel_launcher.py:24: UserWarning: invalid index of a 0-dim tensor. This will be an error in PyTorch 0.5. Use tensor.item() to convert a 0-dim tensor to a Python number


[1/100] Loss: 1.976, Acc: 79.198
[1/100] Loss: 1.972, Acc: 79.318
[1/100] Loss: 1.970, Acc: 79.281
[1/100] Loss: 1.967, Acc: 79.305
[1/100] Loss: 1.965, Acc: 79.231
[1/100] Loss: 1.962, Acc: 79.240
Finish 1 epoch, Loss: 1.961, Acc: 79.297


D:\Anaconda3\envs\torch\lib\site-packages\ipykernel_launcher.py:49: UserWarning: volatile was removed and now has no effect. Use `with torch.no_grad():` instead.
D:\Anaconda3\envs\torch\lib\site-packages\ipykernel_launcher.py:50: UserWarning: volatile was removed and now has no effect. Use `with torch.no_grad():` instead.
D:\Anaconda3\envs\torch\lib\site-packages\ipykernel_launcher.py:53: UserWarning: invalid index of a 0-dim tensor. This will be an error in PyTorch 0.5. Use tensor.item() to convert a 0-dim tensor to a Python number
D:\Anaconda3\envs\torch\lib\site-packages\ipykernel_launcher.py:56: UserWarning: invalid index of a 0-dim tensor. This will be an error in PyTorch 0.5. Use tensor.item() to convert a 0-dim tensor to a Python number


Test Loss: 1.938, Acc: 80.830
Time:8.1 s

**********
epoch 2
[2/100] Loss: 1.944, Acc: 79.406
[2/100] Loss: 1.941, Acc: 79.557
[2/100] Loss: 1.938, Acc: 79.955
[2/100] Loss: 1.936, Acc: 80.190
[2/100] Loss: 1.934, Acc: 80.106
[2/100] Loss: 1.932, Acc: 80.193
Finish 2 epoch, Loss: 1.931, Acc: 80.223
Test Loss: 1.911, Acc: 81.340
Time:8.3 s



In [8]:
float(running_acc)

43264.0

In [10]:
float(running_acc) * 100 / (batch_size * i)

72.10666666666667

In [6]:
i

1875

In [7]:
batch_size

32

In [11]:
out.size()

torch.Size([16, 10])

In [165]:
batch_size * i
print(i)
print(batch_size)
print(len(train_dataset))

1875
32
60000


In [149]:
print('[{}/{}] Loss: {:.6f}, Acc: {:.6f}'.format(
                epoch + 1, num_epoches, running_loss / (batch_size * i),
                running_acc * 100 / (batch_size * i)))

[1/100] Loss: 0.605629, Acc: 86.000000


In [132]:
num_correct.data[0]

D:\Anaconda3\envs\torch\lib\site-packages\ipykernel_launcher.py:1: UserWarning: invalid index of a 0-dim tensor. This will be an error in PyTorch 0.5. Use tensor.item() to convert a 0-dim tensor to a Python number
  """Entry point for launching an IPython kernel.


tensor(27)

In [122]:
torch.max(out, 1)[0].size()
torch.max(out, 1)[1].size()
torch.max(out, 1)

(tensor([1.4354, 1.8158, 2.8642, 2.3265, 4.3354, 1.9226, 3.4508, 2.3127, 0.8361,
         3.0646, 4.3123, 3.7371, 1.4075, 1.2359, 3.4304, 2.6911, 1.3671, 3.1406,
         2.0034, 3.4352, 2.6527, 1.4568, 4.8898, 3.9453, 2.8083, 2.4804, 2.4915,
         1.0021, 0.8441, 3.1524, 2.6410, 2.0503], grad_fn=<MaxBackward0>),
 tensor([2, 8, 4, 8, 2, 5, 0, 3, 4, 1, 3, 1, 5, 9, 4, 4, 5, 2, 7, 1, 9, 7, 3, 2,
         8, 1, 2, 3, 5, 6, 9, 5]))

In [125]:
label

tensor([6, 8, 4, 8, 2, 5, 0, 3, 5, 1, 3, 1, 5, 7, 4, 4, 5, 2, 7, 1, 9, 7, 3, 2,
        8, 1, 7, 3, 1, 6, 9, 5])

In [106]:
loss.data[0]

D:\Anaconda3\envs\torch\lib\site-packages\ipykernel_launcher.py:1: UserWarning: invalid index of a 0-dim tensor. This will be an error in PyTorch 0.5. Use tensor.item() to convert a 0-dim tensor to a Python number
  """Entry point for launching an IPython kernel.


tensor(0.9709)

In [74]:
out.size()

torch.Size([32, 10])

In [97]:
label.size()

torch.Size([32, 1])

In [93]:
loss.data[0] * label.size(0)

D:\Anaconda3\envs\torch\lib\site-packages\ipykernel_launcher.py:1: UserWarning: invalid index of a 0-dim tensor. This will be an error in PyTorch 0.5. Use tensor.item() to convert a 0-dim tensor to a Python number
  """Entry point for launching an IPython kernel.


tensor(1.1984, grad_fn=<AliasBackward>)

In [86]:
torch.Tensor.item(loss.data[0])

D:\Anaconda3\envs\torch\lib\site-packages\ipykernel_launcher.py:1: UserWarning: invalid index of a 0-dim tensor. This will be an error in PyTorch 0.5. Use tensor.item() to convert a 0-dim tensor to a Python number
  """Entry point for launching an IPython kernel.


1.198410987854004

In [103]:
torch.Tensor.item(label)

ValueError: only one element tensors can be converted to Python scalars

In [160]:
a = 100000/3
print('{:.6f}'.format(a))

33333.333333


In [169]:
train_loader.data

AttributeError: 'DataLoader' object has no attribute 'data'

In [11]:
x1 = torch.rand(5,3)
x2 = torch.rand(5,1)

x1 = Variable(x1)
x2 = Variable(x2)

x = torch.cat((x1,x2),1)
print(x)
print(x.size())

tensor([[0.6980, 0.3198, 0.0602, 0.9657],
        [0.9098, 0.6337, 0.6436, 0.2291],
        [0.6896, 0.1961, 0.1484, 0.5837],
        [0.4911, 0.1785, 0.0305, 0.9112],
        [0.0156, 0.3448, 0.5134, 0.3972]])
torch.Size([5, 4])
